# Modeling wooden doors and windows
## Windows
Let's start with windows. Here the reference:
![Reference](http://www.sierrascalemodels.com/Images_Art/windows.gif)

In [1]:
from pyplasm import *
import itertools

# Let's generate a simple dataset
X = [0, 0.02, 0.12, 0.62, 0.72, 0.74]
Y = [0, 0.1, 0.8, 0.82, 0.92]
Z = [0, 0.1]
occupancy = [
    [[True] ,[False],[True],[False]], # x0
    [[True] ,[True] ,[True],[True] ], # x1
    [[True] ,[False],[True],[True]], # x2
    [[True] ,[True] ,[True],[True] ], # x3
    [[True] ,[False],[True],[False]]  # x4
]

Evaluating fenvs.py..
...fenvs.py imported in 0.02 seconds


In [2]:
# Here a first version of the window function. It is a second order function
#  that accepts as params the quote vectors X,Y,Z, the occupancy matrix and
#  as paramters of the returned function the size of the bounding box
def window(X,Y,Z,occupancy):
    
    def genModel(dx,dy,dz):
        cartesianProd = itertools.product(range(len(X)-1),range(len(Y)-1),range(len(Z)-1))
        return STRUCT([
            T([1,2,3])([X[i],Y[j],Z[k]])(
                CUBOID([
                        X[i+1]-X[i],
                        Y[j+1]-Y[j],
                        Z[k+1]-Z[k]
                    ])
            )
            for [i,j,k] in cartesianProd if occupancy[i][j][k]
        ])
    
    return genModel

VIEW(window(X,Y,Z,occupancy)(0,0,0))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x4058870> >

![First Window](./images/first_window.png)
Looks pretty basic, but it works. Now let's define a new version of the function where the dx,dy,dz params actually do something:

In [3]:
# THIS FUNCTION IS BROKEN. IT WORKED, BUT NOW IT WORKS NO MORE.
# This generates a window with the horizontal frame pieces longer of one meter 
#  and the vertical ones of two meters. 
#  Note: 
#   An "horizontal frame piece" is a cell with a X:Y dimensional ratio bigger than 2.
#   A "vertical" one have the X:Y ratio less than 0.5
def window(X,Y,Z,occupancy):
    
    def sum(a):
        def s(b):
            return a+b
        return s
    
    def genModel(dx,dy,dz):
        cartesianProd = [(i,j,k) for (i,j,k) in itertools.product(range(len(X)-1),range(len(Y)-1),range(len(Z)-1))]
        
        for (i,j,k) in cartesianProd:
            
            dimX, dimY, dimZ = X[i+1]-X[i], Y[j+1]-Y[j], Z[k+1]-Z[k]
            
            if occupancy[i][j][k]:
                ratio = dimX / dimY
                if ratio > 3:
                    X1 = X[:i+1] + AA(sum(1))(X[i+1:])  
                elif ratio < 0.33:
                    Y1 = Y[:j+1] + AA(sum(2))(Y[j+1:])
        
        return STRUCT([
            T([1,2,3])([X1[i],Y1[j],Z[k]])(
                CUBOID([
                        X1[i+1]-X1[i],
                        Y1[j+1]-Y1[j],
                        Z[k+1]-Z[k]
                    ])
            )
            for [i,j,k] in cartesianProd if occupancy[i][j][k]
        ])
        
    
    return genModel

VIEW(window(X,Y,Z,occupancy)(0,0,0))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x4058f00> >

In [14]:
# This approach works but maybe it's better to generate three list of indices:
#  - One made by the indices of the "horizontal frame pieces"
#  - One made by the indices of the "vertical frame pieces"
#  - One made by the indices of the other pieces

def window(X,Y,Z,occupancy):
    
    def sum(a):
        def s(b):
            return a+b
        return s
    
    def genModel(dx,dy,dz):
        cartesianProd = itertools.product(range(len(X)-1),range(len(Y)-1),range(len(Z)-1))
        
        # The three lists
        pieces = [(i,j,k) for (i,j,k) in cartesianProd if occupancy[i][j][k]]
        hPieces = [(i,j,k) for (i,j,k) in pieces if (Y[j+1]-Y[j]) > 0.05 and (X[i+1]-X[i]) / (Y[j+1]-Y[j]) > 2]
        vPieces = [(i,j,k) for (i,j,k) in pieces if (X[i+1]-X[i]) > 0.05 and (X[i+1]-X[i]) / (Y[j+1]-Y[j]) < 0.5]
        
        # Now I want to find a set of X quote indices of horizontal frame pieces.
        hPiecesIndexes = list(set([i for (i,_,_) in hPieces]))
        vPiecesIndexes = list(set([j for (_,j,_) in vPieces]))
        
        # Now the length of these pieces
        hPiecesLength = SUM([X[i+1]-X[i] for i in hPiecesIndexes])
        vPiecesLength = SUM([Y[j+1]-Y[j] for j in vPiecesIndexes])
        
        # Now the length that the frame pieces must be
        hPiecesDesiredLength = (dx - X[-1] + hPiecesLength) / len(hPiecesIndexes)
        vPiecesDesiredLength = (dy - Y[-1] + vPiecesLength) / len(vPiecesIndexes)
        
        # Now let's generate the new quote vectors
        finalX = X[:]
        finalY = Y[:]
        
        for i in hPiecesIndexes:
            finalX = finalX[:i+1] + AA(sum(hPiecesDesiredLength-(finalX[i+1]-finalX[i])))(finalX[i+1:])
        
        for j in vPiecesIndexes:
            finalY = finalY[:j+1] + AA(sum(vPiecesDesiredLength-(finalY[j+1]-finalY[j])))(finalY[j+1:])
        
        return STRUCT([
            T([1,2,3])([finalX[i],finalY[j],Z[k]])(
                CUBOID([
                        finalX[i+1]-finalX[i],
                        finalY[j+1]-finalY[j],
                        Z[k+1]-Z[k]
                    ])
            )
            for [i,j,k] in pieces
        ])
        
    
    return genModel


VIEW(window(X,Y,Z,occupancy)(1,2,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x5d20d20> >

![Final Window](./images/window_final.png)

In [15]:
# Ok It works. Now let's test it on more complex data:
X2 = [0, 0.02, 0.12, 0.36, 0.38, 0.62, 0.72, 0.74]
Y2 = [0, 0.1, 0.42, 0.44, 0.8, 0.82, 0.92]
Z2 = [0, 0.1]
occupancy2 = [
    [[True] ,[False],[False],[False],[True],[False]], # x0
    [[True] ,[True] ,[True] ,[True] ,[True],[True] ], # x1
    [[True] ,[False],[True] ,[False],[True],[True]], # x2
    [[True] ,[True] ,[True] ,[True] ,[True],[True]], # x3
    [[True] ,[False],[True] ,[False],[True],[True]], # x4
    [[True] ,[True] ,[True] ,[True] ,[True],[True] ], # x5
    [[True] ,[False],[False],[False],[True],[False]]  # x6
]
VIEW(window(X2,Y2,Z2,occupancy2)(1,2,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x52d5ab0> >

![Window 1](./images/window1.png)
Cool. Let's make an another model using the Z-coords:

In [16]:
X3 = [0, 0.02, 0.12, 0.36, 0.38, 0.62, 0.72, 0.74]
Y3 = [0, 0.1, 0.42, 0.44, 0.8, 0.82, 0.92]
Z3 = [0, 0.04, 0.06, 0.1, 0.12]
F,V = False,True
occupancy3 = [
    [[V,V,V,V],[F,F,F,F],[F,F,F,F],[F,F,F,F],[V,V,V,V],[F,F,F,F]], # x0
    [[V,V,V,V],[V,V,V,F],[V,V,V,F],[V,V,V,F],[V,V,V,V],[V,V,V,F]], # x1
    [[V,V,V,V],[F,F,F,F],[F,V,F,F],[F,F,F,F],[V,V,V,V],[V,V,V,F]], # x2
    [[V,V,V,V],[F,V,F,F],[F,V,F,F],[F,V,F,F],[V,V,V,V],[V,V,V,F]], # x3
    [[V,V,V,V],[F,F,F,F],[F,V,F,F],[F,F,F,F],[V,V,V,V],[V,V,V,F]], # x4
    [[V,V,V,V],[V,V,V,F],[V,V,V,F],[V,V,V,F],[V,V,V,V],[V,V,V,F]], # x5
    [[V,V,V,V],[F,F,F,F],[F,F,F,F],[F,F,F,F],[V,V,V,V],[F,F,F,F]]  # x6
]
VIEW(window(X3,Y3,Z3,occupancy3)(1,2,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x52c3600> >

![Window 2](./images/window2.png)
It does look good. Even if the dz parameter does not have any effect, the function can be considered completed.
#### Here some silly experiments:

In [21]:
VIEW(window(X3,Y3,Z3,occupancy3)(0.5,0.5,1))
VIEW(window(X3,Y3,Z3,occupancy3)(2,2,1))
VIEW(window(X3,Y3,Z3,occupancy3)(0.3,2,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x52c3ae0> >

![Silly 1](./images/silly1.png)
![Silly 2](./images/silly2.png)
![Silly 3](./images/silly3.png)